<a href="https://colab.research.google.com/github/AjayAJ2000/Blog/blob/main/Synthetic_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sdv

In [ ]:
import pandas as pd
df = pd.read_csv("/content/sample_data/MOCK_DATA.csv")
df.head()
df.fillna({'Transplant Center':'Unknown'}, inplace=True)

In [ ]:
from sdv.metadata import Metadata
data_dict = {'data': df}
metadata = Metadata.detect_from_dataframes(data_dict)

In [ ]:
metadata.update_column(
    table_name='data',
    column_name="Transplant Center",
    sdtype="text",
    pii=False
)
metadata.update_column(
    table_name='data',
    column_name="COI Number",
    sdtype = "id",
    regex_format="^CX[0-9]{8}$")
metadata.update_column(
    table_name='data',
    column_name="Patient Name",
    sdtype = "name",
    pii=False)
metadata.update_column(
    table_name='data',
    column_name="Transplant center Patient ID",
    sdtype = "text",
    pii=False)

In [ ]:
metadata.validate()

In [ ]:
metadata

{
    "tables": {
        "data": {
            "columns": {
                "COI Number": {
                    "sdtype": "id",
                    "regex_format": "^CX[0-9]{8}$"
                },
                "Patient ID": {
                    "sdtype": "numerical"
                },
                "Batch Number": {
                    "sdtype": "unknown",
                    "pii": true
                },
                "Order number": {
                    "sdtype": "numerical"
                },
                "Order Status": {
                    "sdtype": "categorical"
                },
                "Final product status": {
                    "sdtype": "categorical"
                },
                "Re-apheresis ?": {
                    "sdtype": "categorical"
                },
                "Re-Apheresis reason": {
                    "sdtype": "categorical"
                },
                "Previous Cancelled date": {
                    "sdtype": "dateti

In [ ]:
for table_name, table_metadata in metadata.tables.items():
    for column_name, column_metadata in table_metadata.columns.items():
        if 'pii' in column_metadata:

          column_metadata['pii'] = False
          print(column_name,column_metadata)

Batch Number {'sdtype': 'unknown', 'pii': False}
Patient Name {'sdtype': 'name', 'pii': False}
Transplant center Patient ID {'sdtype': 'text', 'pii': False}
Transplant Center {'sdtype': 'text', 'pii': False}
Treatment center State {'sdtype': 'administrative_unit', 'pii': False}
Manufacturer PO Number {'sdtype': 'unknown', 'pii': False}
Payer PO Number {'sdtype': 'unknown', 'pii': False}


In [ ]:
from sdv.multi_table import HMASynthesizer

synthesizer = HMASynthesizer(metadata)
synthesizer.fit(data_dict)
synthetic_data = synthesizer.sample(scale=1000)

Preprocess Tables: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it]



Learning relationships:



Modeling Tables: 100%|██████████| 1/1 [00:11<00:00, 11.45s/it]


In [ ]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(data_dict['data'])
synthetic_data = synthesizer.sample(num_rows=10000)

PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name                                                 Est # of Columns (CTGAN)
Patient ID                                                           11
Order number                                                         11
Order Status                                                         9
Final product status                                                 4
Re-apheresis ?                                                       2
Re-Apheresis reason                                                  4
Previous Cancelled date                                              11
Product ID                                                           3
Product Name                                                         3
Line of Therapy                                                      3
Patient Consented                              

In [ ]:
synthetic_data

,COI Number,Patient ID,Batch Number,Order number,Order Status,Final product status,Re-apheresis ?,Re-Apheresis reason,Previous Cancelled date,Product ID,...,Delay Reason Aph delivery to mfg. site,Mfg. time,Delay Reason - Mfg. time,QC time.1,Delay Reason - QC Time,FP Delivery to TC,Delay Reason - FP Delivery to TC,Delivery to Infusion time,Delay Reason - Infusion,COC Person
0,CX00000022,15689.0,sdv-pii-h42hx,NaN,Donor Matched,NaN,NaN,Patient's Unstable Health,NaN,v2,...,Airport delay,NaN,NaN,#NUM!,NaN,On-time,Final delivery delay,NaN,NaN,NaN
1,CX00000007,15699.0,sdv-pii-g9d1k,1260.0,Infused,NaN,NaN,Patient's Unstable Health,NaN,NaN,...,Airport delay,On-time,NaN,Delayed,NaN,Delayed,NaN,Delayed,NaN,NaN
2,CX00000093,NaN,NaN,5116.0,Infused,In Process,NaN,FP Out-Spec,NaN,NaN,...,Airport delay,On-time,NaN,NaN,NaN,On-time,NaN,Delayed,NaN,Order Completed
3,CX00000041,15759.0,sdv-pii-p4278,5584.0,Manufacturing,In Process,NaN,Aph. Out-Spec,NaN,NaN,...,NaN,Delayed,NaN,Delayed,NaN,On-time,NaN,Delayed,TC Operations,Order Cancelled
4,CX00000049,NaN,sdv-pii-sack5,7847.0,Manufacturing,NaN,NaN,FP Out-Spec,NaN,v2,...,Aph delivery delay,On-time,NaN,On-time,NaN,On-time,NaN,Delayed,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CX00000026,NaN,NaN,8534.0,Infused,In Process,NaN,Patient's Unstable Health,NaN,NaN,...,Courier Pickup Delay,On-time,NaN,NaN,NaN,On-time,Final delivery delay,NaN,NaN,NaN
96,CX00000040,15728.0,NaN,4769.0,NaN,In Process,NaN,Patient's Unstable Health,NaN,NaN,...,NaN,NaN,NaN,#NUM!,NaN,Delayed,NaN,Delayed,NaN,NaN
97,CX00000090,15769.0,NaN,NaN,Infused,Cancelled,NaN,FP Out-Spec,03/18/2023 15:56,v2,...,Airport delay,On-time,NaN,On-time,NaN,Delayed,NaN,Delayed,NaN,NaN
98,CX00000055,15756.0,NaN,NaN,NaN,In Process,Yes,NaN,03/23/2023 00:57,NaN,...,NaN,On-time,NaN,Delayed,NaN,NaN,NaN,Delayed,NaN,NaN


In [ ]:
synthetic_data.to_parquet('/content/sample_data/MOCK_DATA_1.parquet')